In [ ]:
import numpy as np
import tensorflow as tf
import keras
import tensorflow_probability as tfp
from tensorflow.keras.layers import Input, Dense, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Set the input and output shapes
INPUT_SHAPE = (128, 128, 4)  # Input shape of the model
OUTPUT_SHAPE = (128, 128, 1)  # Output shape of the model


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
min_max = '/content/drive/MyDrive/example1/min_max_scale_new.npy'
min, max = np.load(min_max, encoding='bytes')
print(min, max)

-236.2857142857143 295.5714285714286


In [ ]:
# Load and preprocess the data
def load_data(file_path):
    data = np.load(file_path)
    return data

def preprocess_data(file_list):
    # data = []
    # labels = []
    loaded_data = []
    for file_name in file_list:
        file_path = '/content/drive/MyDrive/example1/' + file_name
        loaded_data.append(load_data(file_path))
    return np.array(loaded_data)


def generate_data(data,min_train,max_train):
    # data shape=(n_samples, row, col, timesteps)
    n_samples=data.shape[0]
    time_step=data.shape[3]
    row=128
    col=128

    # replace the pixel of "no echo (-127)" as 0
    data[data<=-127]=0

    # for training dataset (t-30,t-20,t-10,t,1+10)
    n_frames=4
    movie_in=np.zeros((n_samples,row,col,n_frames))
    movie_out=np.zeros((n_samples,row,col,1))
    for i in range(n_samples):
        for j in range(n_frames):
            m_in=(255.*((data[i,::,::,j]+10.)/70.))+0.5
            movie_in[i,::,::,j]=m_in
        m_out=(255.*((data[i,::,::,-1]+10.)/70.))+0.5
        movie_out[i,::,::,0]=m_out

    # Min-max scaling
    movie_in=(movie_in-min_train)/(max_train-min_train)
    movie_out=(movie_out-min_train)/(max_train-min_train)

    return movie_in, movie_out


In [ ]:
# Load the training and testing data
train_file_list = np.loadtxt('/content/drive/MyDrive/example1/radar_events_train.txt', dtype=str)
test_file_list = np.loadtxt('/content/drive/MyDrive/example1/radar_events_test.txt', dtype=str)

data_train = preprocess_data(train_file_list)
data_test = preprocess_data(test_file_list)


x_train, y_train = generate_data(data_train, min, max)
x_test, y_test = generate_data(data_test, min, max)

In [ ]:
import tensorflow as tf
from keras.models import *
from keras.layers import *
from keras.optimizers import Adam

def u_net_model(input_shape=(128,128,4)):

    inputs = Input(input_shape)


    conv1s = Conv2D(128, 3, padding='same', kernel_initializer='he_normal')(inputs)
    bn1s = BatchNormalization()(conv1s)
    act1s=Activation('relu')(bn1s)
    pool1 = MaxPooling2D(pool_size=(2, 2))(act1s)
    drop1=Dropout(0.5)(pool1)

    conv2f = Conv2D(256, 3, padding='same', kernel_initializer='he_normal')(drop1)
    bn2f = BatchNormalization()(conv2f)
    act2f=Activation('relu')(bn2f)
    conv2s = Conv2D(512, 3, padding='same', kernel_initializer='he_normal')(act2f)
    bn2s = BatchNormalization()(conv2s)
    act2s=Activation('relu')(bn2s)
    pool2 = MaxPooling2D(pool_size=(2, 2))(act2s)
    drop2=Dropout(0.5)(pool2)

    conv3f = Conv2D(1024, 3, padding='same', kernel_initializer='he_normal')(drop2)
    bn3f = BatchNormalization()(conv3f)
    act3f=Activation('relu')(bn3f)
    drop3 = Dropout(0.5)(act3f)

    up4 = concatenate([UpSampling2D(size=(2, 2))(drop3), act2s], axis=3)
    conv4f = Conv2D(512, 3, padding='same', kernel_initializer='he_normal')(up4)
    bn4f = BatchNormalization()(conv4f)
    act4f=Activation('relu')(bn4f)
    drop4f=Dropout(0.5)(act4f)
    conv4 = Conv2D(256, 3, padding='same',activation='relu', kernel_initializer='he_normal')(drop4f)
    bn4 = BatchNormalization()(conv4)
    act4=Activation('relu')(bn4)

    up5 = concatenate([UpSampling2D(size=(2, 2))(act4), act1s], axis=3)
    conv5f = Conv2D(128, 3, padding='same', kernel_initializer='he_normal')(up5)
    bn5f = BatchNormalization()(conv5f)
    act5f=Activation('relu')(bn5f)
    drop5f=Dropout(0.5)(act5f)
    conv5s = Conv2D(64, 3, padding='same', kernel_initializer='he_normal')(drop5f)
    bn5s = BatchNormalization()(conv5s)
    act5s=Activation('relu')(bn5s)
    drop5s=Dropout(0.5)(act5s)
    conv5 = Conv2D(2, 3, padding='same', kernel_initializer='he_normal')(drop5s)
    bn5 = BatchNormalization()(conv5)
    act5=Activation('relu')(bn5)

    outputs = Conv2D(1, 1, activation='linear')(act5)

    model = Model(inputs=inputs, outputs=outputs)
    return model
# Create an instance of the model
model=u_net_model(input_shape=(128, 128, 4))

# Print the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 4  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 12  4736        ['input_1[0][0]']                
                                8)                                                                
                                                                                                  
 batch_normalization (BatchNorm  (None, 128, 128, 12  512        ['conv2d[0][0]']                 
 alization)                     8)                                                            

In [ ]:
model.compile(loss='binary_crossentropy')

In [ ]:
# Train the model
model.fit(x_train, y_train, batch_size=8, epochs=60)

Epoch 1/60
7/7 [==============================] - 36s 1s/step - loss: 8.7699
Epoch 2/60
7/7 [==============================] - 2s 345ms/step - loss: 8.7699
Epoch 3/60
7/7 [==============================] - 2s 345ms/step - loss: 8.7699
Epoch 4/60
7/7 [==============================] - 2s 346ms/step - loss: 8.7699
Epoch 5/60
7/7 [==============================] - 2s 348ms/step - loss: 8.7699
Epoch 6/60
7/7 [==============================] - 2s 348ms/step - loss: 8.7699
Epoch 7/60
7/7 [==============================] - 2s 350ms/step - loss: 8.7699
Epoch 8/60
7/7 [==============================] - 2s 350ms/step - loss: 8.7699
Epoch 9/60
7/7 [==============================] - 3s 352ms/step - loss: 8.7699
Epoch 10/60
7/7 [==============================] - 3s 353ms/step - loss: 8.7699
Epoch 11/60
7/7 [==============================] - 3s 353ms/step - loss: 8.7699
Epoch 12/60
7/7 [==============================] - 3s 356ms/step - loss: 8.7699
Epoch 13/60
7/7 [==============================] - 

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

epochs = 100
patience = 50

# Create the EarlyStopping callback
early_stopping = EarlyStopping(monitor='loss', patience=patience)

# Train the model
history = model.fit(x_train, y_train, batch_size=8, epochs=epochs, callbacks=[early_stopping])

# Access the training history
loss_values = history.history['loss']

Epoch 1/100
7/7 [==============================] - 3s 355ms/step - loss: 8.7699
Epoch 2/100
7/7 [==============================] - 3s 353ms/step - loss: 8.7699
Epoch 3/100
7/7 [==============================] - 3s 353ms/step - loss: 8.7699
Epoch 4/100
7/7 [==============================] - 3s 353ms/step - loss: 8.7699
Epoch 5/100
7/7 [==============================] - 3s 354ms/step - loss: 8.7699
Epoch 6/100
7/7 [==============================] - 3s 353ms/step - loss: 8.7699
Epoch 7/100
7/7 [==============================] - 3s 353ms/step - loss: 8.7699
Epoch 8/100
7/7 [==============================] - 3s 352ms/step - loss: 8.7699
Epoch 9/100
7/7 [==============================] - 3s 353ms/step - loss: 8.7699
Epoch 10/100
7/7 [==============================] - 3s 352ms/step - loss: 8.7699
Epoch 11/100
7/7 [==============================] - 3s 352ms/step - loss: 8.7699
Epoch 12/100
7/7 [==============================] - 3s 353ms/step - loss: 8.7699
Epoch 13/100
7/7 [===================

In [ ]:
# Evaluate the model on test data
loss = model.evaluate(x_test, y_test, verbose=0)
print(loss)

9.90571403503418


In [ ]:
predictions= model.predict(x_test)

1/1 [==============================] - 0s 239ms/step


In [ ]:
# Save the trained model
model.save('u_net_model.h5')